In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.request

import time
import pandas as pd
import scipy as sc
import numpy as np
import re

import statsmodels.formula.api as sm

import matplotlib.pyplot as plt 
plt.style.use('ggplot')
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6) 

In [2]:
#gets the dataframe from the csv, cleans supressed data
def getgrades(csv1):
    grades1=pd.read_csv(csv1, encoding="utf8")
    #grades2=pd.read_csv(csv2, encoding="utf8")
    #grds=[grades1, grades2]
    #grades=pd.concat(grds,ignore_index=True)
    i=0
    #grades=grades.reset_index(drop=True)
    hc=grades1['sumHeadcount']
    while i < len(hc):
        if hc[i]== 'ds':
            grades1=grades1.drop(i, axis=0)
        i+=1
    grades=grades1.reset_index(drop=True)
    return(grades)

In [3]:
#puts all of the grades for each class on one line
def combinegrades(grades):
    i=1
    n=2
    gradesclean=pd.DataFrame([],columns=['Cat. #', 'Sec.', 'Subject', 'A', 'B', 'C', 'D', 'E', 'W', 'Other'])
    gradesclean=gradesclean.append({'Cat. #':grades['Catalog Num'][0], 'Sec.':grades['Section'][0], 'Subject':grades['Subject'][0], 'A':0, 'B':0, 'C':0, 'D':0,
                                    'E':0, 'W':0, 'Other':0}, ignore_index=True)
    gradesclean[grades['Grade Group'][0]][0]=grades['sumHeadcount'][0]
    while i<len(grades['sumHeadcount']):
        if grades['Subject'].iloc[i]==gradesclean['Subject'].iloc[-1]:
            if grades['Catalog Num'].iloc[i]==gradesclean['Cat. #'].iloc[-1]:
                fail=0
                for m in range(1,n):
                    if grades['Section'].iloc[i]==gradesclean['Sec.'].iloc[-m]:
                        gradesclean[grades['Grade Group'].iloc[i]].iat[-m]=grades['sumHeadcount'].iloc[i]
                        break
                    fail+=1
                if fail==n-1:
                    n+=1
                    gradesclean=gradesclean.append({'Cat. #':grades['Catalog Num'].iloc[i],
                                                    'Sec.':grades['Section'].iloc[i],
                                                    'Subject':grades['Subject'].iloc[i], 'A':0, 'B':0,
                                                    'C':0,'D':0, 'E':0, 'W':0, 'Other':0}, ignore_index=True)
                    gradesclean[grades['Grade Group'].iloc[i]].iat[-1]=grades['sumHeadcount'].iloc[i]
            else:
                n=2
                gradesclean=gradesclean.append({'Cat. #':grades['Catalog Num'].iloc[i], 'Sec.':grades['Section'].iloc[i],
                                                'Subject':grades['Subject'].iloc[i], 'A':0, 'B':0,
                                                'C':0,'D':0, 'E':0, 'W':0, 'Other':0}, ignore_index=True)
                gradesclean[grades['Grade Group'].iloc[i]].iat[-1]=grades['sumHeadcount'].iloc[i]
        else:
            n=2
            gradesclean=gradesclean.append({'Cat. #':grades['Catalog Num'].iloc[i], 'Sec.':grades['Section'].iloc[i],
                                            'Subject':grades['Subject'].iloc[i], 'A':0, 'B':0,
                                            'C':0,'D':0, 'E':0, 'W':0, 'Other':0}, ignore_index=True)
            gradesclean[grades['Grade Group'].iloc[i]].iat[-1]=grades['sumHeadcount'].iloc[i]
        i+=1
    return(gradesclean)

In [4]:
#only code that needs to be edited to add new data
egrades=['MEN F17.csv','MEN S17.csv','MEN F16.csv','MEN S16.csv','MEN F15.csv','MEN S15.csv','MEN F14.csv','MEN S14.csv','MEN F13.csv']
i=0
cleaned=[0,0,0,0,0,0,0,0,0]
while i<len(egrades):
    gradess=getgrades(egrades[i])
    gradescleans=combinegrades(gradess)
    cleaned[i]=gradescleans
    i+=1


In [5]:
#combines the instructor frame and the grades frame, removes classes taught by graduate students
def instruct(excel, working):
    F17t=pd.read_excel(excel)
    i=0
    working['Instructor']= ['ds']*len(working['A'])
    while i<len(working['A']):
        m=0
        while m<len(F17t['Sec.']):
            if working['Cat. #'].iloc[i] == F17t['Cat. #'].iloc[m]:
                if working['Sec.'].iloc[i] == F17t['Sec.'].iloc[m]:
                    working.at[i,'Instructor']= F17t['Instructor'].iloc[m].partition(',')[0]
            m+=1
        i+=1
    i=0
    while i<len(working['A']):
        if working['Instructor'].iloc[i] == 'ds':
            working=working.drop(i, axis=0)
        i+=1
    working=working.reset_index(drop=True)
    return(working)

In [6]:
#only code that needs to be edited to add more data
excels=['F17_meen_classes.xlsx','S17_meen_classes.xlsx','F16_meen_classes.xlsx',
        'S16_meen_classes.xlsx','F15_meen_classes.xlsx','S15_meen_classes.xlsx',
        'F14_meen_classes.xlsx','S14_meen_classes.xlsx','F13_meen_classes.xlsx']
insts=[]
i=0
while i<9:
    insts.append(instruct(excels[1],cleaned[1]))
    i+=1
insts[0]['Sem']= ['F17']*len(insts[0]['A'])
insts[1]['Sem']= ['S17']*len(insts[1]['A'])
insts[2]['Sem']= ['F16']*len(insts[2]['A'])
insts[3]['Sem']= ['S16']*len(insts[3]['A'])
insts[4]['Sem']= ['F15']*len(insts[4]['A'])
insts[5]['Sem']= ['S15']*len(insts[5]['A'])
insts[6]['Sem']= ['F14']*len(insts[6]['A'])
insts[7]['Sem']= ['S14']*len(insts[7]['A'])
insts[8]['Sem']= ['F13']*len(insts[8]['A'])
finished=pd.concat(insts,ignore_index=True)
mask=finished['Instructor'] != 'ds'
finished=finished[mask]
finished=finished.reset_index(drop=True)

In [7]:
#takes the h-index list and combines it with the instrutors and grades, removes instructors that don't have a h-index
hscore=[('ABBOTT', 28), ('ACEVDO', 8), ('AMEEL', 27), ('BRANNON', 17),
 ('CALAF', 9), ('CHANG', 11), ('COATS', 12), ('CZABAJ', 9),
 ('FRANCOEUR', 21), ('FU', 17), ('GALE', 34), ('KINGSTEDT', 4),
 ('KONG', 6), ('LEANG', 31), ('LENZI', 22), ('MASCARO', 17),
 ('MCMURTRY', 26), ('MEEK', 21), ('MERRYWEATHER', 11), ('MINOR', 21),
 ('MONSON', 12), ('NALEWAY', 10), ('NEWELL', 8), ('PARDYJAK', 26), 
 ('PARK', 18), ('RAEYMAEKERS', 15), ('ROUNDY', 20), ('SMITH', 10),
 ('SPEAR', 5), ('STOLL', 11), ('TAN', 7), ('WARREN', 6)]
i=0
finished['h-score']= ['ds']*len(finished['A'])
while i < len(finished['Sem']):
    for m in hscore:
        if m[0] == finished['Instructor'].iloc[i]:
            finished.at[i,'h-score']= m[1]
    i+=1
mask=finished['h-score'] != 'ds'
finished=finished[mask]
finished=finished.reset_index(drop=True)
finished

,Cat. #,Sec.,Subject,A,B,C,D,E,W,Other,Instructor,Sem,h-score
0,1000,1,ME EN - Mechanical Engineering,32,8,2,1,1,0,0,CHANG,F17,11
1,1010,1,ME EN - Mechanical Engineering,56,40,21,10,4,11,0,MASCARO,F17,17
2,2010,1,ME EN - Mechanical Engineering,34,43,28,30,9,7,1,COATS,F17,12
3,2960,1,ME EN - Mechanical Engineering,5,0,0,0,1,0,0,SMITH,F17,10
4,2960,3,ME EN - Mechanical Engineering,6,0,0,0,0,0,0,SMITH,F17,10
5,3000,1,ME EN - Mechanical Engineering,26,41,29,8,14,0,0,RAEYMAEKERS,F17,15
6,3210,1,ME EN - Mechanical Engineering,24,74,30,1,0,0,1,MINOR,F17,21
7,3220,1,ME EN - Mechanical Engineering,13,42,9,2,3,0,0,MINOR,F17,21
8,3300,1,ME EN - Mechanical Engineering,16,37,13,1,6,1,1,NEWELL,F17,8
9,3600,1,ME EN - Mechanical Engineering,17,25,24,6,7,1,0,PARK,F17,18


In [8]:
#csv for use in other code
finished.to_csv('h-scored.csv')